<a href="https://colab.research.google.com/github/Subhrayan/A-small-scale-iterable-workflow-to-analyze-sentiment-of-news-paper-articles/blob/main/Urban_gardeing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd

API_KEY = '9cd7e974b9d8d96e4bae9de8e965491d'
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather?'

def get_weather_data(city):
    url = f"{BASE_URL}q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city_name': city,
        'avg_temperature': data['main']['temp'],
        'avg_humidity': data['main']['humidity'],
        'avg_rainfall': data['rain']['1h'] if 'rain' in data else 0,
        'avg_sunlight': data['sys']['sunset'] - data['sys']['sunrise']
    }

cities = ['New York', 'London', 'Tokyo', 'Paris', 'Sydney']  # Add more cities as needed
weather_data = [get_weather_data(city) for city in cities]
df = pd.DataFrame(weather_data)
df.to_csv('climate_data.csv', index=False)

KeyError: 'main'

In [18]:
import requests
import json
import pandas as pd

API_KEY = '9cd7e974b9d8d96e4bae9de8e965491d'
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather?'

def get_weather_data(city):
    url = f"{BASE_URL}q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return {
            'city_name': city,
            'avg_temperature': data['main']['temp'] if 'main' in data else None, # Handle potential missing 'main' key
            'min_temperature': data['main']['temp_min'] if 'main' in data else None,
            'max_temperature': data['main']['temp_max'] if 'main' in data else None,
            'avg_humidity': data['main']['humidity'] if 'main' in data else None,
            'avg_rainfall': data['rain']['1h'] if 'rain' in data else 0,
            'avg_sunlight': data['sys']['sunset'] - data['sys']['sunrise'] if 'sys' in data else None,
            'cloudiness': data['clouds']['all'] if 'clouds' in data else None,
            'wind_speed': data['wind']['speed'] if 'wind' in data else None,
            'probability_of_precipitation': data['pop'] if 'pop' in data else None


        }
    else:
        print(f"Error fetching data for {city}: {response.status_code}")
        print(response.json()) # Print the API response for debugging
        return None

cities_df = pd.read_csv('cleaned_world_cities.csv')
cities = cities_df['name'].tolist()
 # Add more cities as needed
weather_data = [get_weather_data(city) for city in cities]

# Filter out any None values (failed API calls)
weather_data = [data for data in weather_data if data is not None]

df = pd.DataFrame(weather_data)
df.to_csv('climate_data.csv', index=False)

Streaming output truncated to the last 5000 lines.
Error fetching data for Marblehead: 429
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'}
Error fetching data for Marlborough: 429
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'}
Error fetching data for Medford: 429
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'}
Error fetching data for Melrose: 429
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'

In [ ]:
import requests
import json
import pandas as pd
import time
from datetime import datetime

API_KEY = 'e82b7f5627266a841c316dea317477c6'
BASE_URL = 'http://api.openweathermap.org/data/2.5/weather?'

def get_weather_data(city):
    url = f"{BASE_URL}q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return {
            'city_name': city,
            'avg_temperature': data['main']['temp'],
            'min_temperature': data['main']['temp_min'] if 'main' in data else None,
            'max_temperature': data['main']['temp_max'] if 'main' in data else None,
            'avg_humidity': data['main']['humidity'],
            'avg_rainfall': data['rain']['1h'] if 'rain' in data and '1h' in data['rain'] else 0,
            'avg_sunlight': data['sys']['sunset'] - data['sys']['sunrise'],
            'date': datetime.now().strftime('%Y-%m-%d')
        }
    else:
        print(f"Failed to get data for {city}: {response.status_code}")
        return None

def collect_weather_data(cities):
    weather_data = []
    for i, city in enumerate(cities, 1):
        data = get_weather_data(city)
        if data:
            weather_data.append(data)

        # Check if we've made 60 requests, and if so, wait until the next minute
        if i % 60 == 0:
            print(f"Processed {i} cities. Waiting for the next minute to respect API rate limit...")
            time.sleep(61)  # Wait for 61 seconds to ensure we're in the next minute

    return weather_data

# Load cities from a CSV file
cities_df = pd.read_csv('world_cities.csv')
cities = cities_df['name'].tolist()

# Collect weather data
weather_data = collect_weather_data(cities)

# Create DataFrame and save to CSV
df = pd.DataFrame(weather_data)
df.to_csv('climate_data.csv', index=False)

print(f"Data collection complete. Collected data for {len(weather_data)} cities.")

In [22]:
import requests
import json
import pandas as pd
import time
from datetime import datetime, timedelta

API_KEY = '9cd7e974b9d8d96e4bae9de8e965491d'
BASE_URL = 'https://history.openweathermap.org/data/2.5/aggregated/year?'

def get_weather_data(city, lat, lon):
    url = f"{BASE_URL}lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return {
            'city_name': city,
            'latitude': lat,
            'longitude': lon,
            'avg_temperature': data['result']['temp']['mean'],
            'avg_humidity': data['result']['humidity']['mean'],
            'avg_rainfall': data['result']['precipitation']['mean'],
            'record_date': datetime.now().strftime('%Y-%m-%d')
        }
    else:
        print(f"Failed to get data for {city}: {response.status_code}")
        return None

def get_cities(filename='cleaned_world_cities.csv', limit=100):
    df = pd.read_csv(filename)
    return df[['name', 'latitude', 'longitude']].head(limit).to_dict('records')

def collect_weather_data(cities):
    weather_data = []
    for i, city in enumerate(cities):
        if i > 0 and i % 60 == 0:
            print("Waiting for 60 seconds due to API rate limit...")
            time.sleep(60)

        data = get_weather_data(city['name'], city['latitude'], city['longitude'])
        if data:
            weather_data.append(data)

        print(f"Processed {i+1}/{len(cities)} cities")

    return weather_data

# Main execution
cities = get_cities(limit=100)  # Adjust limit as needed
weather_data = collect_weather_data(cities)

df = pd.DataFrame(weather_data)
df.to_csv('climate_data.csv', index=False)
print("Data collection complete. Results saved to climate_data.csv")

Failed to get data for les Escaldes: 401
Processed 1/100 cities
Failed to get data for Andorra la Vella: 401
Processed 2/100 cities
Failed to get data for Warīsān: 401
Processed 3/100 cities
Failed to get data for Umm Al Quwain City: 401
Processed 4/100 cities
Failed to get data for Ras Al Khaimah City: 401
Processed 5/100 cities
Failed to get data for Zayed City: 401
Processed 6/100 cities
Failed to get data for Khawr Fakkān: 401
Processed 7/100 cities
Failed to get data for Kalbā: 401
Processed 8/100 cities
Failed to get data for Jumayrā: 401
Processed 9/100 cities
Failed to get data for Dubai: 401
Processed 10/100 cities
Failed to get data for Dibba Al-Fujairah: 401
Processed 11/100 cities
Failed to get data for Dibba Al-Hisn: 401
Processed 12/100 cities
Failed to get data for Sharjah: 401
Processed 13/100 cities
Failed to get data for Ar Ruways: 401
Processed 14/100 cities
Failed to get data for Al Fujairah City: 401
Processed 15/100 cities
Failed to get data for Al Ain City: 401
P

In [25]:
!pip install meteostat

import pandas as pd
from meteostat import Point, Daily
from datetime import datetime

def get_climate_data(city, lat, lon):
    # Create Point for the location
    location = Point(lat, lon)

    # Set time period
    start = datetime(2020, 1, 1)
    end = datetime(2020, 12, 31)

    # Fetch daily data for the year 2020
    data = Daily(location, start, end)
    data = data.fetch()

    # Calculate averages

    avg_temp = data['tavg'].mean()
    min_temp = data['tmin'].mean()
    max_temp = data['tmax'].mean()

    avg_precipitation = data['prcp'].mean()
    avg_wind_speed = data['wspd'].mean()
    avg_sunshine_minutes = data['tsun'].mean()

    return {
        'city_name': city,
        'latitude': lat,
        'longitude': lon,
        'avg_temperature': avg_temp,
        'min_temperature': min_temp,
        'max_temperature': max_temp,

        'avg_rainfall': avg_precipitation,
        'avg_wind_speed': avg_wind_speed,
        'avg_sunshine_minutes': avg_sunshine_minutes
    }

def get_cities(filename='cleaned_world_cities.csv', limit=100):
    df = pd.read_csv(filename)
    return df[['name', 'latitude', 'longitude']].head(limit).to_dict('records')

def collect_climate_data(cities):
    climate_data = []
    for i, city in enumerate(cities):
        data = get_climate_data(city['name'], city['latitude'], city['longitude'])
        if data:
            climate_data.append(data)

        print(f"Processed {i+1}/{len(cities)} cities")

    return climate_data

# Main execution
cities = get_cities(limit=100)  # Adjust limit as needed
climate_data = collect_climate_data(cities)

df = pd.DataFrame(climate_data)
df.to_csv('climate_data_meteostat.csv', index=False)
print("Data collection complete. Results saved to climate_data.csv")

Processed 1/100 cities
Processed 2/100 cities
Processed 3/100 cities
Processed 4/100 cities
Processed 5/100 cities


Processed 6/100 cities
Processed 7/100 cities
Processed 8/100 cities
Processed 9/100 cities
Processed 10/100 cities
Processed 11/100 cities
Processed 12/100 cities
Processed 13/100 cities
Processed 14/100 cities
Processed 15/100 cities
Processed 16/100 cities
Processed 17/100 cities
Processed 18/100 cities
Processed 19/100 cities
Processed 20/100 cities
Processed 21/100 cities
Processed 22/100 cities
Processed 23/100 cities
Processed 24/100 cities
Processed 25/100 cities
Processed 26/100 cities
Processed 27/100 cities
Processed 28/100 cities
Processed 29/100 cities
Processed 30/100 cities
Processed 31/100 cities
Processed 32/100 cities
Processed 33/100 cities
Processed 34/100 cities


Processed 35/100 cities
Processed 36/100 cities
Processed 37/100 cities
Processed 38/100 cities
Processed 39/100 cities
Processed 40/100 cities
Processed 41/100 cities
Processed 42/100 cities
Processed 43/100 cities
Processed 44/100 cities
Processed 45/100 cities
Processed 46/100 cities
Processed 47/100 cities


Processed 48/100 cities
Processed 49/100 cities
Processed 50/100 cities
Processed 51/100 cities
Processed 52/100 cities
Processed 53/100 cities
Processed 54/100 cities
Processed 55/100 cities
Processed 56/100 cities
Processed 57/100 cities
Processed 58/100 cities
Processed 59/100 cities
Processed 60/100 cities
Processed 61/100 cities
Processed 62/100 cities
Processed 63/100 cities


Processed 64/100 cities
Processed 65/100 cities
Processed 66/100 cities
Processed 67/100 cities
Processed 68/100 cities
Processed 69/100 cities
Processed 70/100 cities
Processed 71/100 cities
Processed 72/100 cities
Processed 73/100 cities
Processed 74/100 cities
Processed 75/100 cities
Processed 76/100 cities
Processed 77/100 cities
Processed 78/100 cities
Processed 79/100 cities
Processed 80/100 cities
Processed 81/100 cities
Processed 82/100 cities
Processed 83/100 cities


Processed 84/100 cities
Processed 85/100 cities
Processed 86/100 cities
Processed 87/100 cities
Processed 88/100 cities
Processed 89/100 cities
Processed 90/100 cities
Processed 91/100 cities
Processed 92/100 cities
Processed 93/100 cities
Processed 94/100 cities
Processed 95/100 cities
Processed 96/100 cities
Processed 97/100 cities
Processed 98/100 cities
Processed 99/100 cities
Processed 100/100 cities
Data collection complete. Results saved to climate_data.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import rasterio
import pandas as pd

def get_climate_data(city, lat, lon):
    # Paths to WorldClim data files (you need to download these)
    temp_file = 'wc2.1_10m_tavg_01.tif'  # Average temperature for January
    prec_file = 'wc2.1_10m_prec_01.tif'  # Precipitation for January

    with rasterio.open(temp_file) as temp_src, rasterio.open(prec_file) as prec_src:
        temp_val = next(temp_src.sample([(lon, lat)]))[0]
        prec_val = next(prec_src.sample([(lon, lat)]))[0]

    return {
        'city_name': city,
        'latitude': lat,
        'longitude': lon,
        'avg_temperature': temp_val,
        'avg_rainfall': prec_val,
    }

# The rest of the script (get_cities and collect_climate_data functions)
# can remain the same as in the previous examples

In [19]:
import requests
import json
import pandas as pd
import gzip
from io import BytesIO

def get_city_list():
    url = 'http://bulk.openweathermap.org/sample/city.list.json.gz'
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception(f"Failed to download file: Status code {response.status_code}")

    # Decompress the gzip file
    with gzip.GzipFile(fileobj=BytesIO(response.content)) as gzipfile:
        data = json.loads(gzipfile.read().decode('utf-8'))

    return data

def process_city_data(data):
    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Ensure 'population' column exists and is numeric
    if 'population' not in df.columns:
        print("Warning: 'population' column not found in the data.")
        return pd.DataFrame()

    df['population'] = pd.to_numeric(df['population'], errors='coerce')

    # Filter cities with population > 50,000
    df_filtered = df[df['population'] >= 50000].copy()

    # Select and rename relevant columns
    df_filtered = df_filtered.rename(columns={
        'name': 'city_name',
        'coord.lat': 'latitude',
        'coord.lon': 'longitude',
        'country': 'country_code'
    })

    columns_to_keep = ['city_name', 'latitude', 'longitude', 'country_code', 'population']
    df_filtered = df_filtered[columns_to_keep]

    return df_filtered

def main():
    try:
        print("Downloading city data...")
        city_data = get_city_list()

        print("Processing city data...")
        df = process_city_data(city_data)

        if df.empty:
            print("No cities found matching the criteria.")
            return

        print(f"Found {len(df)} cities with population > 50,000")

        # Save to CSV
        output_file = 'openweathermap_cities_50k.csv'
        df.to_csv(output_file, index=False)
        print(f"Data saved to {output_file}")

        # Display first few rows
        print("\nFirst few rows of the data:")
        print(df.head())

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Processing city data...
No cities found matching the criteria.


In [7]:
import pandas as pd
import requests
from io import BytesIO
from zipfile import ZipFile

# Download and extract the data
url = 'http://download.geonames.org/export/dump/cities15000.zip'
response = requests.get(url)
zip_file = ZipFile(BytesIO(response.content))
csv_file = zip_file.open('cities15000.txt')

# Read the data into a pandas DataFrame
columns = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature_class', 'feature_code', 'country_code', 'cc2', 'admin1_code', 'admin2_code', 'admin3_code', 'admin4_code', 'population', 'elevation', 'dem', 'timezone', 'modification_date']
df = pd.read_csv(csv_file, sep='\t', names=columns, low_memory=False)

# Select relevant columns and save to CSV
cities_df = df[['name', 'latitude', 'longitude', 'country_code', 'population']]
cities_df.to_csv('world_cities.csv', index=False)

In [8]:
import requests
import json
import pandas as pd

url = 'http://bulk.openweathermap.org/sample/city.list.json.gz'
response = requests.get(url)

# Decompress the gzip file
import gzip
from io import BytesIO

with gzip.GzipFile(fileobj=BytesIO(response.content)) as gzipfile:
    data = json.loads(gzipfile.read().decode('utf-8'))

# Convert to DataFrame
df = pd.DataFrame(data)
df.to_csv('openweathermap_cities.csv', index=False)

In [11]:
!pip install cities

import cities

# Get all cities
all_cities = list(cities.cities)

# Create a DataFrame
import pandas as pd

cities_df = pd.DataFrame(all_cities, columns=['name', 'lat', 'lng', 'country', 'admin'])
cities_df.to_csv('world_cities_from_library.csv', index=False)

ModuleNotFoundError: No module named 'cities'

In [12]:
!pip install cities # Install the cities module

import cities

# Rename the 'cities' variable to avoid conflict
city_list = list(cities.cities)

# Create a DataFrame
import pandas as pd

cities_df = pd.DataFrame(city_list, columns=['name', 'lat', 'lng', 'country', 'admin'])
cities_df.to_csv('world_cities_from_library.csv', index=False)

ModuleNotFoundError: No module named 'cities'

In [13]:
!pip install citipy # Install the citipy module

import citipy

# Rename the 'cities' variable to avoid conflict
city_list = list(citipy.cities)

# Create a DataFrame
import pandas as pd

cities_df = pd.DataFrame(city_list, columns=['name', 'lat', 'lng', 'country', 'admin'])
cities_df.to_csv('world_cities_from_library.csv', index=False)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 675.6/675.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for citipy: filename=citipy-0.0.6-py3-none-any.whl size=675966 sha256=4aedf7efb39c23029d8a3ddd8454c67b747181bc8e56834be2b5ac2b978cdfb8
  Stored in directory: /root/.cache/pip/wheels/41/82/37/8bb56372b0d828d226b83e0117780ecbe9f409100ac52364bd
Successfully built citipy


AttributeError: module 'citipy' has no attribute 'cities'

In [15]:
import pandas as pd

# Load data from different sources
geonames_df = pd.read_csv('world_cities.csv')

openweather_df = pd.read_csv('openweathermap_cities.csv')

# Combine dataframes
combined_df = pd.concat([geonames_df, openweather_df], ignore_index=True)

# Remove duplicates
combined_df.drop_duplicates(subset=['name', 'latitude', 'longitude'], keep='first', inplace=True)

# Save the combined list
combined_df.to_csv('combined_world_cities.csv', index=False)

<ipython-input-15-7a6fe8ae20da>:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  openweather_df = pd.read_csv('openweathermap_cities.csv')


In [16]:
# Remove cities with missing data
combined_df.dropna(subset=['name', 'latitude', 'longitude'], inplace=True)

# Validate latitude and longitude
combined_df = combined_df[(combined_df['latitude'] >= -90) & (combined_df['latitude'] <= 90)]
combined_df = combined_df[(combined_df['longitude'] >= -180) & (combined_df['longitude'] <= 180)]

# Remove potential duplicates with slightly different coordinates
combined_df = combined_df.round({'latitude': 2, 'longitude': 2})
combined_df.drop_duplicates(subset=['name', 'latitude', 'longitude'], keep='first', inplace=True)

# Save the cleaned data
combined_df.to_csv('cleaned_world_cities.csv', index=False)

In [1]:
import zipfile

!unzip /content/wc2.1_30s_vapr.zip

Archive:  /content/wc2.1_30s_vapr.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/wc2.1_30s_vapr.zip or
        /content/wc2.1_30s_vapr.zip.zip, and cannot find /content/wc2.1_30s_vapr.zip.ZIP, period.
